In [ ]:
from quantities import UnitQuantity, m, nm, Hz, MHz, s, kg, eV, J, C, K, N, A

from sympy import symbols, integrate, solve, latex, oo, simplify
import sympy as sy
from IPython.display import display, Math

from numpy import linspace, exp, e, pi, sqrt, sinh, imag, sin, vectorize, where, delete, absolute
import numpy as np

from plotly.express import scatter
from pandas import DataFrame

def Units(x, units):
    x.units = units
    return x

def split_off_units(x, decimal_places = None):
    if decimal_places is not None:
        x = round(x, decimal_places)
    x = str(x)
    return f'{x.split()[0]} \\text{{{x.split()[1]} }}'

epsilon0 = 8.854187817 * 10**(-12) * C**2/(N*m**2)
mu0 = 4*pi*10**(-7) * N/(A**2)
c = 1/(mu0*epsilon0)**0.5

In [ ]:
# 1.

Show that the commutator relation $\left[\vec{r},{\vec{p}}^2\right]=2i\hbar\vec{p}$ holds true. (434: 10 pts, 591: 6 pts)

# 2.

Like what we have done for the Harmonic Oscillator, we can define a ladder operator for the angular momentum: $L_\pm=L_x\pm\ iL_y$. The benefit of such a ladder operator is that we can generate the higher (and lower) order wave functions by applying the respective ladder operator. The ladder operators for the angular momentum change the value of the quantum number m of the Spherical Harmonics Y_l^mby one unit:

$$L_\pm Y_l^m=\hbar\sqrt{l\left(l+1\right)-m(m\pm1)}\ Y_l^{m\pm1}$$

### a. 

Using the equations for the Spherical Harmonic function for $Y_1^0$, show that the ladder operators reproduce the Spherical Harmonic functions for $Y_1^{-1}$ and $Y_1^1$, respectively. 

### b. 

Plot all three functions (or their absolute square) separately to visualize them. (all: 20 pts + 10 pts bonus)

Caution, this is a very lengthy problem. If you use external sources to help you out, please cite them.

You can have a look at the Wikipedia page for the Spherical Harmonics: https://en.wikipedia.org/wiki/Spherical_harmonics 


# 3.

Determine the wavelengths of the three lowest Lyman and Balmer transitions of the Hydrogen atom. (434: 10 pts, 591: 6 pts)

Hint: These series are important in Astronomy and in Optical Spectroscopy, since a lot of UV-VIS spectrometers use Deuterium arc lamps.

Answer from answer sheet:

Lyman 2 – 1 : λ = 121.6 nm.


# 4.

How large is the energy shift of the $l=1$ energy level of a Hydrogen atom in an external magnetic field of $B = 0.3\text{T}$, assuming that there is only an interaction with the angular magnetic moment? (434: 10 pts, 591: 6 pts)

Answer from answer sheet:

$E_{\text{shift}} = 9.27\cdot 10^{−24} \text{J} \cdot \text{B} = 5.79\cdot10^{−5} \text{eV} \cdot \text{B}  = 2.78\cdot10^{-24} \text{J} = 1.74\cdot10^{−5} \text{eV}$


# 2.

Like what we have done for the Harmonic Oscillator, we can define a ladder operator for the angular momentum: $L_\pm=L_x\pm\ iL_y$. The benefit of such a ladder operator is that we can generate the higher (and lower) order wave functions by applying the respective ladder operator. The ladder operators for the angular momentum change the value of the quantum number m of the Spherical Harmonics Y_l^mby one unit:

$$L_\pm Y_l^m=\hbar\sqrt{l\left(l+1\right)-m(m\pm1)}\ Y_l^{m\pm1}$$

### a. 

Using the equations for the Spherical Harmonic function for $Y_1^0$, show that the ladder operators reproduce the Spherical Harmonic functions for $Y_1^{-1}$ and $Y_1^1$, respectively. 

### b. 

Plot all three functions (or their absolute square) separately to visualize them. (all: 20 pts + 10 pts bonus)

Caution, this is a very lengthy problem. If you use external sources to help you out, please cite them.

You can have a look at the Wikipedia page for the Spherical Harmonics: https://en.wikipedia.org/wiki/Spherical_harmonics 


# 3.

Determine the wavelengths of the three lowest Lyman and Balmer transitions of the Hydrogen atom. (434: 10 pts, 591: 6 pts)

Hint: These series are important in Astronomy and in Optical Spectroscopy, since a lot of UV-VIS spectrometers use Deuterium arc lamps.

Answer from answer sheet:

Lyman 2 – 1 : λ = 121.6 nm.


# 4.

How large is the energy shift of the $l=1$ energy level of a Hydrogen atom in an external magnetic field of $B = 0.3\text{T}$, assuming that there is only an interaction with the angular magnetic moment? (434: 10 pts, 591: 6 pts)

Answer from answer sheet:

$E_{\text{shift}} = 9.27\cdot 10^{−24} \text{J} \cdot \text{B} = 5.79\cdot10^{−5} \text{eV} \cdot \text{B}  = 2.78\cdot10^{-24} \text{J} = 1.74\cdot10^{−5} \text{eV}$
